<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-pandas-and-NumPy,-as-well-as-the-os-module" data-toc-modified-id="Import-pandas-and-NumPy,-as-well-as-the-os-module-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import pandas and NumPy, as well as the os module</a></span></li><li><span><a href="#Load-the-data-from-Cameroon-and-Poland" data-toc-modified-id="Load-the-data-from-Cameroon-and-Poland-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load the data from Cameroon and Poland</a></span></li><li><span><a href="#Concatenate-the-Cameroon-and-Poland-data" data-toc-modified-id="Concatenate-the-Cameroon-and-Poland-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Concatenate the Cameroon and Poland data</a></span></li><li><span><a href="#Concatenate-all-the-country-data-files" data-toc-modified-id="Concatenate-all-the-country-data-files-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Concatenate all the country data files</a></span></li><li><span><a href="#Show-some-of-the-combined-data" data-toc-modified-id="Show-some-of-the-combined-data-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Show some of the combined data</a></span></li><li><span><a href="#Check-the-values-in-the-concatenated-data" data-toc-modified-id="Check-the-values-in-the-concatenated-data-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Check the values in the concatenated data</a></span></li><li><span><a href="#Fix-the-missing-values" data-toc-modified-id="Fix-the-missing-values-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Fix the missing values</a></span></li></ul></div>

# Import pandas and NumPy, as well as the os module

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# pd.set_option('display.width', 200)
# pd.set_option('display.max_columns', 35)
# pd.set_option('display.max_rows', 50)
# pd.options.display.float_format = '{:,.0f}'.format

In [3]:
import watermark
%load_ext watermark

%watermark -n -i -iv

pandas   : 1.2.1
json     : 2.0.9
numpy    : 1.19.2
watermark: 2.1.0



# Load the data from Cameroon and Poland

In [4]:
ltcameroon = pd.read_csv('data/ltcountry/ltcameroon.csv')
ltpoland = pd.read_csv('data/ltcountry/ltpoland.csv')

# Concatenate the Cameroon and Poland data

In [5]:
ltcameroon.shape

(48, 11)

In [6]:
ltpoland.shape

(120, 11)

In [7]:
ltpoland.columns

Index(['locationid', 'year', 'month', 'temperature', 'latitude', 'longitude',
       'elevation', 'station', 'countryid', 'country', 'latabs'],
      dtype='object')

In [8]:
ltcameroon.columns

Index(['locationid', 'year', 'month', 'temperature', 'latitude', 'longitude',
       'elevation', 'station', 'countryid', 'country', 'latabs'],
      dtype='object')

In [9]:
ltall = pd.concat([ltcameroon, ltpoland])

In [10]:
ltall['country'].value_counts()

Poland      120
Cameroon     48
Name: country, dtype: int64

# Concatenate all the country data files

In [11]:
directory = 'data/ltcountry'
ltall = pd.DataFrame()

In [12]:
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        fileloc = os.path.join(directory, filename)
        # open the next file
        with open(fileloc) as file:
            ltnew = pd.read_csv(fileloc)
            print(filename + ' has ' + str(ltnew.shape[0]) + ' rows.')
            ltall = pd.concat([ltall, ltnew])
            # check for differences in columns
            columndiff = ltall.columns.symmetric_difference(ltnew.columns)
            if (not columndiff.empty):
                print('',
                      'Different column names for: ',
                      filename,
                      columndiff,
                      sep='\n')

ltbrazil.csv has 1104 rows.
ltcameroon.csv has 48 rows.
ltindia.csv has 1056 rows.
ltjapan.csv has 1800 rows.
ltmexico.csv has 852 rows.
ltoman.csv has 288 rows.

Different column names for: 
ltoman.csv
Index(['latabs'], dtype='object')
ltpoland.csv has 120 rows.


# Show some of the combined data

In [13]:
ltall[['country', 'station', 'month', 'temperature',
       'latitude']].sample(5, random_state=1)

,country,station,month,temperature,latitude
717,Brazil,TAGUATINGA,8,27.75,-12.400
649,Japan,MATSUMOTO,5,17.25,36.250
172,Oman,BURAIMI_AUT,8,37.40,24.233
1029,India,JAIPUR_SANGANER,12,14.92,26.817
351,Mexico,SN_CRISTOBAL_LAS_CASASCHIS,5,19.30,16.750


# Check the values in the concatenated data

In [14]:
ltall['country'].value_counts().sort_index()

Brazil      1104
Cameroon      48
India       1056
Japan       1800
Mexico       852
Oman         288
Poland       120
Name: country, dtype: int64

In [15]:
ltall.groupby(['country']).agg({
    'temperature': ['min', 'mean', 'max', 'count'],
    'latabs': ['min', 'mean', 'max', 'count']
})

temperature                          latabs                         
                 min       mean    max count     min       mean     max count
country                                                                      
Brazil         12.20  25.470196  33.93   969   0.051  13.859272  33.517  1104
Cameroon       21.87  27.227941  36.00    34   4.020   7.789250  10.451    48
India           1.95  26.113477  37.40  1044   8.300  21.104516  34.083  1056
Japan          -7.15  15.495854  30.14  1797  24.300  35.664822  45.417  1800
Mexico          7.00  23.230397  34.45   806  14.917  22.114901  32.400   852
Oman           12.10  27.753463  38.39   205     NaN        NaN     NaN     0
Poland         -3.85  10.462750  23.25   120  50.078  52.498500  54.750   120

# Fix the missing values

In [16]:
ltall['latabs'] = np.where(ltall['country'] == 'Oman', ltall['latitude'],
                           ltall['latabs'])

In [17]:
ltall.groupby(['country']).agg({
    'temperature': ['min', 'mean', 'max', 'count'],
    'latabs': ['min', 'mean', 'max', 'count']
})

temperature                          latabs                         
                 min       mean    max count     min       mean     max count
country                                                                      
Brazil         12.20  25.470196  33.93   969   0.051  13.859272  33.517  1104
Cameroon       21.87  27.227941  36.00    34   4.020   7.789250  10.451    48
India           1.95  26.113477  37.40  1044   8.300  21.104516  34.083  1056
Japan          -7.15  15.495854  30.14  1797  24.300  35.664822  45.417  1800
Mexico          7.00  23.230397  34.45   806  14.917  22.114901  32.400   852
Oman           12.10  27.753463  38.39   205  16.933  21.997875  26.167   288
Poland         -3.85  10.462750  23.25   120  50.078  52.498500  54.750   120